In [1]:
import pandas as pd
import mba263
import matplotlib.pyplot as plt
import os

In [21]:
df = pd.read_csv(os.path.dirname(os.getcwd()) + "/data/BBB.csv")
df

,acctnum,gender,state,zip,zip3,first,last,book_,nonbook_,total_,purch,child,youth,cook,do_it,refernce,art,geog,buyer
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,no
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,no
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,no
3,10004,F,NJ,7016,70,7,7,15,257,272,1,0,0,0,0,1,0,0,no
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27,1,1,0,0,0,0,0,0,no
49996,59997,F,NJ,8882,88,25,5,79,294,373,7,3,0,1,1,0,1,1,no
49997,59998,M,NJ,7410,74,3,3,15,178,193,1,0,0,0,0,1,0,0,no
49998,59999,M,NJ,7090,70,49,29,98,246,344,8,2,0,1,0,2,1,2,no


\newpage

# Part 1: Logistic Regression

## 1. Estimate a logistic regression model using "buyer" as the dependent variable.

In [ ]:
# create dummy variables for buyer and gender variables
df['buyer_int'] = pd.get_dummies(df['buyer'])['yes']
df['female'] = pd.get_dummies(df['gender'])['F']
df

In [ ]:
# use logit to predict logistic regression probabilities
result = mba263.logit(df['buyer_int'], df[['last', 'total_', 'female', 'child', 'youth', 'cook', 'do_it', 'refernce', 'art', 'geog']])
result.predict()

In [ ]:
df['buy_prob'] = result.predict()
df

\newpage

## 2. Summarize and interpret the results (so that a marketing manager can understand them). Which variables are significant? Which seem to be 'important'? Interpret the odds-ratios for each of the predictors. 

In [ ]:
result.summary()

In [ ]:
mba263.odds_ratios(result)

Analyzing the summary of the Logistic Regression results, we can see that all coefficients are significant because all of their p-values are less than 0.05 However, their impacts on the probability of being a buyer all vary, some being very helpful while others not being very helpful. To analysze which ones are more impactful, we look at the coefficient's odds ratio.


The way odds-ratio works is for every unit of increase in a certain coefficient's category, we multiply the probability of being a buyer by that coefficient's odds ratio. Looking at the odds ratio of our Logistic Regression, we can see that a handful of coefficients have an odds ratio of less than 1. Specifically, ```last```, ```female```, ```child```, ```youth```, ```cook```, and ```do_it``` all have odds ratios less than 1. What this means is that for every unit of increase in any of those categories separately, $P(buyer)$ actually ends up decreasing. For example, for an increase of 1 ```youth``` book purchase by a customer, the probability of them buying from this catalog is multiplied by 0.893, which decreases the probability of them purchasing from our catalog according to the Logistic Regression Model. Since each of these categories have a odds ratio of less than 1, they will end up decreasing our $P(buyer)$ if the number in those categories increases. Therefore, while they are significant in our model, they are not important.

```total_``` has an odds ratio of about 1, so a unit of increase in a customer's total spending increases the probability of them buying from thsi catalog by a factor of 1, which is not really that helpful. ```total_``` is a significant coefficient, but not really important since it doesn't really increase $P(buyer)$

The important coefficients here are ```refernce```, ```art```, and ```geog```. They all have an odds ratio of greater than 1. This means that for every unit of increase in each of these categories, the probability of a customer purchasing from this catalog increases by a factor of 1.26, 3.17, and 1.77, respectively. Since these ratios are greater than one, the probability will increase for every additional unit purchased in eqach respective category. Therefore, these are the important variables that we should focus on.

\newpage

# Part 2: Decile Analysis of Logistic Regression Results

## 1. Assign each customer to a decile based on his or her predicted probability of purchase.

In [ ]:
# doing 9 - calculation to make it so the lower the decile number, the higher the probability of purchasing
df['prob_decile'] = 9 - mba263.ntile(df['buy_prob'], 10)
df

\newpage

## 2. Create a bar chart plotting response rate by decile (as just defined above).

In [ ]:
df[['prob_decile', 'buyer_int']].groupby('prob_decile').mean()

In [ ]:
df[['prob_decile', 'buyer_int']].groupby('prob_decile').mean().plot(kind = 'bar')
plt.ylabel('response_rate');

\newpage

## 3. Generate a report showing number of customers, the number of buyers of "The Art History of Florence' and the response rate to the offer by decile for the random sample (i.e. the 50,000 customers) in the dataset. 

In [ ]:
temp_df = df[['prob_decile', 'buyer', 'buyer_int']]
temp_df['buyer_int_2'] = df['buyer_int']
report_df = temp_df[['prob_decile', 'buyer', 'buyer_int', 'buyer_int_2']].groupby('prob_decile').agg({'buyer' : 'count', 'buyer_int' : 'sum', 'buyer_int_2' : 'mean'})
report_df.rename(columns = {"buyer" : "num_customers", "buyer_int": "num_buyers", "buyer_int_2":"response_rate"}, inplace = True)
report_df

\newpage

## 4. For the 50,000 customers in the dataset, generate a report showing the mean values of the following variables by probability of purchase decile:
- Total $ spent 
- Months since last purchase
- Number of books purchased for each of the seven categories (i.e., children, youth, cookbooks, do-it-yourself, reference, art and geography). 

In [ ]:
mean_values = df[['total_', 'last', 'child', 'youth', 'cook', 'do_it', 'refernce', 'art', 'geog', 'prob_decile']].groupby('prob_decile').mean()
mean_values

\newpage

## 5. Summarize and interpret the decile analysis results. Are the patterns in the decile analysis consistent with your conclusions from the logistic regression?

Yes, the patterns in the decile analysis is consisten with my conclusions from the logistic regression, specifically when analyzing odds ratio. When analyzing odds ratio, I found out that the important variables to look at were ```refernce```, ```art```, and ```geog```, since they all had an odds ratio of greater than 1. Looking at the mean values based on probability of purchase deciles, we can see as the decile number increases, the mean values for these three categories decrease. On the other hand, the mean value for all the other categories stay relatively in the same range for each decile, which makes them not really a good predictor of the probability of a customer being a buyer. This was also given in our odds ratio analysis, since these variables (```last```, ```child```, ```youth```, ```cook```, and ```do_it```) were not important according to our analysis.

In our logistic regression analysis, we saw that art had the highest odds ratio of 3.17, meaning it was the most impactful variable in predicting the probability of a customer being a buyer. This meant that as a customer purchased more art books in the past, the probability of them being a buyer increased by a factor of 3.17. Looking at the mean analysis per decile, we can see the top decile has the highest mean of art books purchased, being 1.5. As the deciles increase, the mean of art books purchased decreases. Those in decile 0 had the highest mean of art books purchased, which our odds ratio predicts that they have a higher probability of responding to our catalog. Looking at the decile response rate grah, we can see that decile 0 had the highest response rate to our catalog, which shows that our decile analysis is consistent with our logistic regression analysis.

\newpage

# Part 3: Lifts and Gains

## 1. Use the information from the report in II.3 above to create a table showing the lift and cumulative lift for each decile. You may want to use Excel for these calculations.

In [ ]:
lift_report = report_df[['num_customers', 'num_buyers']]
num_responded = 4522
num_sample = 50000
overall_response_rate = num_responded / num_sample

customers_per_decile = lift_report['num_customers'].tolist()
buyers_per_decile = lift_report['num_buyers'].tolist()

lift_report['cum. cust'] = [sum(customers_per_decile[0:i]) for i in range(1, len(customers_per_decile) + 1)]
lift_report['cum. % cust'] = (lift_report['cum. cust'] / num_sample)

lift_report['cum. buy'] = [sum(buyers_per_decile[0:i]) for i in range(1, len(buyers_per_decile) + 1)]
lift_report['cum. % buy'] = (lift_report['cum. buy'] / num_responded)

lift_report = lift_report[['num_customers', 'cum. cust', 'cum. % cust', 'num_buyers', 'cum. buy', 'cum. % buy']]

In [ ]:
# lift calculations
lift_report['response rate'] = (lift_report['num_buyers'] / lift_report['num_customers'])
lift_report['lift'] = (lift_report['response rate'] / overall_response_rate) * 100

lift_report['cum. response rate'] = (lift_report['cum. buy'] / lift_report['cum. cust'])
lift_report['cum. lift'] = (lift_report['cum. response rate'] / overall_response_rate) * 100

lift_report

In [ ]:
mba263.lift(df['buyer_int'], df['buy_prob'])

\newpage

## 2. Create a chart showing the cumulative lift by decile.

In [ ]:
plt.plot(lift_report['cum. lift'])
plt.xlabel('deciles')
plt.ylabel('cumulative lift')
plt.title('cumulative lift by decile')
plt.xticks([i for i in range(10)]);

\newpage

## 3. Use the information from the report in II.3 above to create a table showing the gains and cumulative gains for each decile. You may want to use Excel for these calculations.

In [ ]:
gains_report = lift_report[['num_customers', 'cum. cust', 'cum. % cust', 'num_buyers', 'cum. buy']]
gains_report['gains'] = gains_report['num_buyers'] / num_responded
gains_report['cum. gains'] = [sum(gains_report['gains'].tolist()[0:i]) for i in range(1, len(gains_report['gains'].tolist()) + 1)]

gains_report

In [ ]:
mba263.gain(df['buyer_int'], df['buy_prob'])

\newpage

## 4. Create a chart showing the cumulative gains by decile along with a reference line corresponding to 'no model'.

In [ ]:
plt.plot([0] + gains_report['cum. gains'].tolist(), label = 'gains model')
plt.plot([0, 10], [0, 1], label = 'no model')
plt.legend(loc = "upper left")
plt.xlabel('% customer targeted')
plt.ylabel('probability')
plt.title('cumulative gains')
plt.xticks([i for i in range(11)]);

\newpage

# Part 4: Profitability Analysis

### Use the following cost information to assess the profitability of using logistic regression to determine which of the remaining 500,000 customers should receive a specific offer: 
- Cost to mail offer to customer: \$0.50
- Selling price (shipping included): \$18.00
- Wholesale price paid by BookBinders: \$9.00
- Shipping costs: \$3.00

## 1. What is the breakeven response rate?

In [ ]:
revenue_per_sale = 18
profit_per_sale = revenue_per_sale - 9 - 3
cost_of_mailing = 0.5
break_even_rate = cost_of_mailing / profit_per_sale
print("The breakeven response rate is " + str(break_even_rate * 100) + "%" )

\newpage

## 2. For the customers in the dataset, create a new variable (call it "target") with a value of 1 if the customer's predicted probability is greater than or equal to the breakeven response rate and 0 otherwise. 

In [ ]:
df['target'] = (df['buy_prob'] >= break_even_rate) * 1
df

\newpage

## 3. Considering that there are 500,000 remaining customers, generate a report summarizing the number of customers, the expected number of buyers of 'The Art History of Florence' and the expected response rate to the offer by the "target" variable. 

In [ ]:
# the proportion of the 500,000 remaining customers that will respond to our catalog
prop_mail_to = sum(df['target']) / len(df)
prop_mail_to

In [ ]:
# number of customers expected to mail to
num_remaining_mail_to = 500000 * prop_mail_to
num_remaining_mail_to

In [ ]:
# the expected response rate based on targeted mailing

targeted_response_rate = mba263.tabulate(df[df['target'] == 1]['buyer'])['Frequency'][1]
targeted_response_rate

In [ ]:
# the expected number of buyers based on how many we mail to
num_respond_to = targeted_response_rate * num_remaining_mail_to
num_respond_to

\newpage

## 4. For the 500,000 remaining customers, what would the expected gross profit (in dollars, and also as a percentage of gross sales) and the expected return on marketing expenditures have been if BookBinders had mailed the offer to buy "The Art History of Florence" only to customers with a predicted probability of buying that was greater than or equal to the breakeven rate? 

In [ ]:
gross_profit = profit_per_sale * num_respond_to - cost_of_mailing * num_remaining_mail_to
gross_sales_targeted = revenue_per_sale * num_respond_to

print("The expected gross profit is $" + str(gross_profit))
print("The expected gross profit as a percentage of gross sales is " + str((gross_profit / gross_sales_targeted) * 100) + "%") 
print("The expected return on marketing expenditures is " + str(gross_profit / (cost_of_mailing * num_remaining_mail_to) * 100) + "%")